In [ ]:
!apt-get -qq update && apt-get -qq install -y  libgdcm-tools python-gdcm
!conda install -y -q -c conda-forge opencv gdcm pydicom apache-beam -y

In [ ]:
import numpy as np
import tensorflow as tf
import apache_beam
import pydicom
import gdcm
import cv2 as cv
import matplotlib.pyplot as plt
import os
import sys
import csv
import pickle
import random
from datetime import datetime
print('Libraries Imported!')
import matplotlib.pyplot as plt
import PIL
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pandas as pd
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from skimage import exposure
from skimage.transform import resize

np.set_printoptions(threshold=sys.maxsize)
print('Settings Set!')
print(tf.__version__)

In [ ]:
rows = 9
def showDataset(data, num):    
    plt.figure(figsize=(15,15))
    for num, x in enumerate(data):
        plt.subplot(rows,6,num+1)
        plt.xlabel(x[1])
        plt.imshow(x[0], cmap="gray")  

    
def list_dicoms_in_folder(folder):
    '''Lists the full path to all the DICOM files in a given folder.'''
    my_list = []
    i = 0 
    for dirname, _, filenames in os.walk(folder):
        for filename in filenames:
            if filename[-4:]=='.dcm':
                my_list.append([os.path.join(dirname, filename),filename.split('.')[0]])
                i = i +1
        if i ==50:
            break
    return my_list


In [ ]:
# Change this to True if you want to do something with all the data
# For testing purposes, you can just use the small list of files defined in the next cell
TRAIN_FOLDER = '/kaggle/input/siim-covid19-detection/train'
TEST_FOLDER = '/kaggle/input/siim-covid19-detection/test'
VERBOSE = False

if True:
    Train_Data = list_dicoms_in_folder(TRAIN_FOLDER)
    Test_Data = list_dicoms_in_folder(TEST_FOLDER)
    
train_images=[]
test_images=[]

for element,name in Train_Data:
    ds = pydicom.dcmread(element)
    train_images.append([ds.pixel_array,name])

showDataset(train_images, 48)    



In [ ]:
for element,name in Test_Data:
    ds = pydicom.dcmread(element)
    test_images.append([ds.pixel_array,name])

showDataset(test_images, 48)    


Reading training dataset and writing to annotation.txt
Saving dcm images to jpg


In [ ]:
os.mkdir('train')

# Visualising training set

In [ ]:
import ast
import cv2
import numpy as np
import pandas as pd
import csv
i = 0
df = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv')  
df2 = pd.read_csv('/kaggle/input/siim-covid19-detection/train_study_level.csv')  

# create annotation.txt and append values
f = open("annotation.txt","a+")
f.truncate(0)

res = [(row,img) for index,row in df.iterrows() 
        for img,name in train_images if name in row['id']]

f= open("annotation.txt","a+")
f.truncate(0)

# Iterate over image csv rows
for r,img in (res):
    if str(r['boxes'])!='nan':
        
        grp = [c for index,c in df2.iterrows() if r['StudyInstanceUID'] in c['id']]
       
        if grp[0]['Negative for Pneumonia'] ==1:
             classification = '0'
        elif grp[0]['Typical Appearance'] ==1:
             classification = '1'
        elif grp[0]['Indeterminate Appearance'] ==1:
             classification = '2'
        elif grp[0]['Atypical Appearance'] ==1:
             classification = '3'
      
        result= (np.maximum(img,0) / img.max()) * 255.0
        
        cv.imwrite(os.path.join('/kaggle/working/train/', r['id'][:-6]+'.jpg'), result)
        # Read jpeg image
        jpg_img = cv.imread(os.path.join('/kaggle/working/train/', r['id'][:-6]+'.jpg'))
        
        plt.figure(figsize=(5,5))
        plt.xlabel(r['id'][:-6] + ':'+ classification)
        f.write(str(r['id'][:-6])+".jpg ")
        for b in ast.literal_eval(r['boxes']):
            x1= b['x']
            y1 = b['y']
            w1 = b['width'] + b['x']
            h1= b['height'] + b['y']
            
            cv.rectangle(jpg_img, (int(x1), int(y1)), (int(w1), int(h1)), (255, 0, 0), 10)
        
            
            f.write(str(int(x1))+" ")
            f.write(str(int(y1))+" ")
            f.write(str(int(w1))+" ")
            f.write(str(int(h1))+" ")
            f.write(str(classification)+" ")
            plt.imshow(jpg_img)  
        
        f.write("\n")
f.close()         

In [ ]:
YOLO_TYPE                   = "yolov3" 
TRAIN_LOAD_IMAGES_TO_RAM = True
TRAIN_ANNOT_PATH            = "/kaggle/working/annotation.txt"
TEST_ANNOT_PATH             = "/kaggle/working/annotation.txt"
TRAIN_BATCH_SIZE            = 4
TRAIN_INPUT_SIZE            = 416
TEST_BATCH_SIZE             = 4
TEST_INPUT_SIZE             = 416
TRAIN_IMAGES= '/kaggle/working/train'
TEST_IMAGES= '/kaggle/working/test'
YOLO_STRIDES                = [8, 16, 32]
TRAIN_CLASSES               = ['0','1','2','3']
YOLO_ANCHOR_PER_SCALE       = 3
YOLO_MAX_BBOX_PER_SCALE     = 100

YOLO_STRIDES                = [8, 16, 32]
YOLO_IOU_LOSS_THRESH        = 0.5
YOLO_INPUT_SIZE             = 416
if YOLO_TYPE                == "yolov3":
    YOLO_ANCHORS            = [[[10,  13], [16,   30], [33,   23]],
                               [[30,  61], [62,   45], [59,  119]],
                               [[116, 90], [156, 198], [373, 326]]]
YOLO_V3_WEIGHTS             = "/kaggle/input/yolov3-weights"
TRAIN_LOGDIR                = "/kaggle/working/log"
TRAIN_CHECKPOINTS_FOLDER    = "/kaggle/working/checkpoints"
YOLO_COCO_CLASSES           = ['0','1','4','7','3','6','10','2','5','8','9']
TRAIN_CLASSES               = ['0','1','2','3']
TRAIN_MODEL_NAME            = f"{YOLO_TYPE}_custom"
TRAIN_LOAD_IMAGES_TO_RAM    = True # With True faster training, but need more RAM
TRAIN_BATCH_SIZE            = 4
TRAIN_INPUT_SIZE            = 416
TRAIN_DATA_AUG              = True
TRAIN_TRANSFER              = True
TRAIN_FROM_CHECKPOINT       = False # "checkpoints/yolov3_custom"
TRAIN_LR_INIT               = 0.005
TRAIN_LR_END                = 0.009
TRAIN_WARMUP_EPOCHS         = 2
TRAIN_EPOCHS                = 500

In [ ]:

os.mkdir('./log')
os.mkdir('./checkpoints')


In [ ]:
def showDatasetWithBB(img,bboxes,typeOfimage):    
    plt.figure(figsize=(5,5))
    color = (0, 255, 0)
    
    for x in bboxes:
        cv.putText(img, 'gt bbox', (x[0], x[1]), cv.FONT_HERSHEY_DUPLEX, 0.7, color, 4)
        cv.rectangle(img, (x[0], x[1]), (x[0]+x[2],  x[1]+x[3]), color, 7)
        cv.circle(img, (int((x[0]+ x[2])/2), int((x[1]+ x[3])/2)), 3, color, -1)
        print("Type",typeOfimage,"BBbox: ",x)
        
        plt.imshow(img)
        plt.show()
        

In [ ]:
def image_preprocess(image, target_size, gt_boxes=None):
    ih, iw    = target_size
    h,  w, _  = image.shape

    scale = min(iw/w, ih/h)
    nw, nh  = int(scale * w), int(scale * h)
    image_resized = cv.resize(image, (nw, nh))

    image_paded = np.full(shape=[ih, iw, 3], fill_value=128.0)
    dw, dh = (iw - nw) // 2, (ih-nh) // 2
    image_paded[dh:nh+dh, dw:nw+dw, :] = image_resized
    image_paded = image_paded / 255.
    
    showDatasetWithBB(image,gt_boxes,"ground truth boxes")
    
    if gt_boxes is None:
        return image_paded
    else:
        gt_boxes[:, [2, 3]] = np.multiply(gt_boxes[:, [2, 3]] , scale) + dw
        gt_boxes[:, [0, 1]] = np.multiply(gt_boxes[:, [0, 1]] , scale) + dh

    showDatasetWithBB(image_paded,gt_boxes,"preprocessed ground truth boxes")
    return image_paded, gt_boxes


In [ ]:
class Dataset(object):
    # Dataset preprocess implementation
    def __init__(self, dataset_type, TEST_INPUT_SIZE=TEST_INPUT_SIZE):
        
        self.annot_path  = TRAIN_ANNOT_PATH if dataset_type == 'train' else TEST_ANNOT_PATH
        self.input_sizes = TRAIN_INPUT_SIZE if dataset_type == 'train' else TEST_INPUT_SIZE
        self.batch_size  = TRAIN_BATCH_SIZE if dataset_type == 'train' else TEST_BATCH_SIZE
        self.train_input_sizes = TRAIN_INPUT_SIZE
        self.image_path    = TRAIN_IMAGES   if dataset_type == 'train' else TEST_IMAGES
        self.batch_count = 0
        self.strides = np.array(YOLO_STRIDES)
        self.anchors = (np.array(YOLO_ANCHORS).T/self.strides).T
        self.anchor_per_scale = YOLO_ANCHOR_PER_SCALE
        self.max_bbox_per_scale = YOLO_MAX_BBOX_PER_SCALE
        self.classes = (TRAIN_CLASSES)
        self.num_classes = len(self.classes)
        
        self.annotations = self.load_annotations(dataset_type)
        self.num_samples = len(self.annotations)
        self.num_batchs = int(np.ceil(self.num_samples / self.batch_size))
        
    def __iter__(self):
        return self
    
    def __len__(self):
        return self.num_batchs

    def __next__(self):
        self.train_input_size = random.choice([self.train_input_sizes])
        self.train_output_sizes = self.train_input_size // self.strides

        batch_image = np.zeros((self.batch_size, self.train_input_size, self.train_input_size, 3), dtype=np.float32)

        batch_label_sbbox = np.zeros((self.batch_size, self.train_output_sizes[0], self.train_output_sizes[0],
                                      self.anchor_per_scale, 5 + self.num_classes), dtype=np.float32)
        batch_label_mbbox = np.zeros((self.batch_size, self.train_output_sizes[1], self.train_output_sizes[1],
                                      self.anchor_per_scale, 5 + self.num_classes), dtype=np.float32)
        batch_label_lbbox = np.zeros((self.batch_size, self.train_output_sizes[2], self.train_output_sizes[2],
                                      self.anchor_per_scale, 5 + self.num_classes), dtype=np.float32)

        batch_sbboxes = np.zeros((self.batch_size, self.max_bbox_per_scale, 4), dtype=np.float32)
        batch_mbboxes = np.zeros((self.batch_size, self.max_bbox_per_scale, 4), dtype=np.float32)
        batch_lbboxes = np.zeros((self.batch_size, self.max_bbox_per_scale, 4), dtype=np.float32)

        exceptions = False
        num = 0
        if self.batch_count < self.num_batchs:
            while num < self.batch_size:
                index = self.batch_count * self.batch_size + num
                if index >= self.num_samples: index -= self.num_samples
                annotation = self.annotations[index]
                image, bboxes = self.parse_annotation(annotation)
                
                try:
                    label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes = self.preprocess_true_boxes(bboxes)
                except IndexError:
                    exceptions = True
                    continue
            
                batch_image[num, :, :, :] = image
                batch_label_sbbox[num, :, :, :, :] = label_sbbox
                batch_label_mbbox[num, :, :, :, :] = label_mbbox
                batch_label_lbbox[num, :, :, :, :] = label_lbbox
                batch_sbboxes[num, :, :] = sbboxes
                batch_mbboxes[num, :, :] = mbboxes
                batch_lbboxes[num, :, :] = lbboxes
                num += 1
            self.batch_count += 1
            batch_smaller_target = batch_label_sbbox, batch_sbboxes
            batch_medium_target  = batch_label_mbbox, batch_mbboxes
            batch_larger_target  = batch_label_lbbox, batch_lbboxes
            
            return batch_image, (batch_smaller_target, batch_medium_target, batch_larger_target)
        else:
            self.batch_count = 0
            np.random.shuffle(self.annotations)
            raise StopIteration
            
    def load_annotations(self, dataset_type):
        final_annotations = []
        with open(self.annot_path, 'r') as f:
            txt = f.readlines()
            annotations = [line.strip() for line in txt if len(line.strip().split()[1:]) != 0]
           # print(annotations)
            
        np.random.shuffle(annotations)
       
        for annotation in annotations:
            # fully parse annotations
            line = annotation.split(" ")
            #print(line)
            image_path, index = "", 1
            for i, one_line in enumerate(line):
                if not one_line.replace(",","").isnumeric():
                    if image_path != "": image_path += " "
                    image_path += one_line
                else:
                    index = i
                    break
            
            if not os.path.exists(os.path.join(self.image_path,image_path)):
                continue
            if TRAIN_LOAD_IMAGES_TO_RAM:
                image = cv.imread(os.path.join(self.image_path,image_path))
            else:
                image = ''
                
            final_annotations.append([os.path.join(self.image_path,image_path), line[1:], image])
            
        return final_annotations

       
    def parse_annotation(self, annotation, mAP = 'False'):
        if True:
            image_path = annotation[0]
            image = cv.imread(image_path)
        else:
            image_path = os.path.join(TRAIN_IMAGES,annotation[0])
            image = cv.imread(image_path)
        
        i = 0
        box=[]
        bboxes=[]
       # print(annotation[1])
        for box_ind in annotation[1]:
            box.append(int(box_ind))
            i = i + 1 
            if i%5==0:
                bboxes.append(box)
                box= []
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if mAP == True: 
            return image, bboxes
        
     
        image, bboxes = image_preprocess(np.copy(image), [self.input_sizes, self.input_sizes], np.copy(bboxes))
        
        return image, bboxes

    def preprocess_true_boxes(self, bboxes):
        label = [np.zeros((self.train_output_sizes[i], self.train_output_sizes[i], self.anchor_per_scale,
                           5 + self.num_classes)) for i in range(3)]
        bboxes_xywh = [np.zeros((self.max_bbox_per_scale, 4)) for _ in range(3)]
        bbox_count = np.zeros((3,))
        
        for bbox in bboxes:
            bbox_coor = bbox[:4]
            bbox_class_ind = bbox[4]

            onehot = np.zeros(self.num_classes, dtype=np.float)
            onehot[bbox_class_ind] = 1.0
            uniform_distribution = np.full(self.num_classes, 1.0 / self.num_classes)
            deta = 0.01
            smooth_onehot = onehot * (1 - deta) + deta * uniform_distribution

            bbox_xywh = np.concatenate([(bbox_coor[2:] + bbox_coor[:2]) * 0.5, bbox_coor[2:] - bbox_coor[:2]], axis=-1)
            bbox_xywh_scaled = 1.0 * bbox_xywh[np.newaxis, :] / self.strides[:, np.newaxis]

            iou = []
            exist_positive = False
            for i in range(3):
                anchors_xywh = np.zeros((self.anchor_per_scale, 4))
                anchors_xywh[:, 0:2] = np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.int32) + 0.5
                anchors_xywh[:, 2:4] = self.anchors[i]

                iou_scale = bbox_iou(bbox_xywh_scaled[i][np.newaxis, :], anchors_xywh)
                iou.append(iou_scale)
                iou_mask = iou_scale > 0.3

                if np.any(iou_mask):
                    xind, yind = np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.int32)

                    label[i][yind, xind, iou_mask, :] = 0
                    label[i][yind, xind, iou_mask, 0:4] = bbox_xywh
                    label[i][yind, xind, iou_mask, 4:5] = 1.0
                    label[i][yind, xind, iou_mask, 5:] = smooth_onehot

                    bbox_ind = int(bbox_count[i] % self.max_bbox_per_scale)
                    bboxes_xywh[i][bbox_ind, :4] = bbox_xywh
                    bbox_count[i] += 1

                    exist_positive = True

            if not exist_positive:
                best_anchor_ind = np.argmax(np.array(iou).reshape(-1), axis=-1)
                best_detect = int(best_anchor_ind / self.anchor_per_scale)
                best_anchor = int(best_anchor_ind % self.anchor_per_scale)
                xind, yind = np.floor(bbox_xywh_scaled[best_detect, 0:2]).astype(np.int32)

                label[best_detect][yind, xind, best_anchor, :] = 0
                label[best_detect][yind, xind, best_anchor, 0:4] = bbox_xywh
                label[best_detect][yind, xind, best_anchor, 4:5] = 1.0
                label[best_detect][yind, xind, best_anchor, 5:] = smooth_onehot

                bbox_ind = int(bbox_count[best_detect] % self.max_bbox_per_scale)
                bboxes_xywh[best_detect][bbox_ind, :4] = bbox_xywh
                bbox_count[best_detect] += 1
        label_sbbox, label_mbbox, label_lbbox = label
        sbboxes, mbboxes, lbboxes = bboxes_xywh
        return label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes


    def random_horizontal_flip(self, image, bboxes):
      
        return image, bboxes

    def random_crop(self, image, bboxes):
        
        return image, bboxes

    def random_translate(self, image, bboxes):
       
        return image, bboxes
    

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf
import random
#from tensorflow.keras.utils import plot_model
    

In [ ]:
if YOLO_TYPE == "yolov4":
    Darknet_weights = YOLO_V4_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V4_WEIGHTS
if YOLO_TYPE == "yolov3":
    Darknet_weights =  YOLO_V3_WEIGHTS


gpus = tf.config.experimental.list_physical_devices('GPU')
print(f'GPUs {gpus}')
if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)
    
trainset = Dataset('train')
testset = Dataset('test')


In [ ]:
res = [(img,name) for img,name in train_images]

for image,name in res:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image_preprocess(np.copy(image), [TEST_INPUT_SIZE, TEST_INPUT_SIZE])


In [ ]:
res = [(img,name) for img,name in test_images]

for img,name in res:
    result= (np.maximum(img,0) / img.max()) * 255.0
    
    cv.imwrite(os.path.join('/kaggle/working/test/'+name+'.jpg'), result)
    # Read jpeg image
    jpg_img = cv.imread('/kaggle/working/test/'+name+'.jpg')
    plt.figure(figsize=(5,5))
    plt.xlabel(name)
    plt.imshow(result,cmap='gray')